# XML: Fortgeschrittene Themen

In der Vorlesung haben wir einige fortgeschrittene Themen behandelt, wie XML Baumstruktur, Vokabularien mit XML, und XML Programmierung. Hier werden wir diese Themen etwas in der Praxis anschauen.

## XML Baumstruktur

Da jedes XML Dokument genau ein Wurzelelement (*root element*) hat und Kindelement (*child elements*) entsprechend verschachtelt, kann jedes (wohlgeformte) XML Dokument als Baum dargestellt werden. Probieren Sie dies in dieser Übung. Die Funktion `print_tree` wird Ihnen dabei helfen. Schauen Sie sich das folgende XML Dokument an und schreiben Sie dann Ihr eigenes. Wie verändert sich das Verhalten der `print_tree` Funktion wenn die beiden kommentierten Zeilen ausgeführt werden?

In [16]:
def print_tree(xml):
    level = 0
    context = et.iterwalk(xml, events=('start', 'end'))
    for event, element in context:
        if event == 'start':
            print('{}> {}'.format('  ' * level, element.tag))
            if not len(element):
                print('{}> {}'.format('  ' * (level + 1), element.text))
            level = level + 1
        if event == 'end':
            level = level - 1

In [17]:
from lxml import etree as et

doc = """<discography><albums>
<album><title>The Dark Side of the Moon</title><released>16 March 1973</released></album>
<album><title>The Wall</title><released>30 November 1979</released></album>
</albums></discography>
"""

print_tree(et.fromstring(doc))

> discography
  > albums
    > album
      > title
        > The Dark Side of the Moon
      > released
        > 16 March 1973
    > album
      > title
        > The Wall
      > released
        > 30 November 1979


In [18]:
from lxml import etree as et

# My XML document ...
doc = """<burger><brötchen><patties></patties><salat></salat><sauce></sauce><bacon></bacon></brötchen><brötchen></brötchen></burger>

"""

print_tree(et.fromstring(doc))

> burger
  > brötchen
    > patties
      > None
    > salat
      > None
    > sauce
      > None
    > bacon
      > None
  > brötchen
    > None


## Vokabularien in XML

Spannen Sie mit Ihrem Nachbarn zusammen und legen Sie sich auf ein Themenbereich fest, z.B. Adressen, Bücher, oder Personen. Überlegen Sie sich dann einzeln ein XML Vokabular für den gewählten Themenbereich. Welche Elemente benötigt man um den Themenbereich zu beschreiben? 

Im folgenden Block schreiben Sie dann ein beispielhaftes XML Dokument. Diskutieren Sie danach mit Ihrem Nachbarn Ihr gewähltes Vokabular und das Beispiel. Bestimmt gibt es Unterschiede! Haben die beiden erstellten Vokabulare unterschiedliche Terme? Wurden diese als *Tag* oder *Attribut* modelliert? 

Diskutieren Sie mit Ihrem Nachbarn die Vor- und Nachteile der Vokabularien und versuchen Sie sich zu einigen.

**Hinweis:** Die Übung können Sie auch von zuhause ausführen, indem Sie mit Kommiliton*innen über Email kommunizieren. Arbeitskollegen sitzen nicht immer im gleichen Raum!

**My vocabulary for Personen**

<personen>
Geschlecht
Alter
Größe
Haarfarbe
Augenfarbe
Gewicht
Schuhgröße
</personen>

## XML in der Programmierung

XML Daten können programmatisch aus den verschiedesten Sourcen gelesen werden. String Variablen hatten wir in den Übungen schon mehrmals. XML Daten können selbstverständlich auch aus Dateien oder dem Internet gelesen werden. Das folgende Beispiel liest die `example.xml` Datei welche im gleichen Verzeichnis wie dieses Notebook liegt.

In [1]:
from lxml import etree as et

doc = et.parse('example.xml')

print(et.tostring(doc, pretty_print=True).decode('utf-8'))

OSError: Error reading file 'example.xml': failed to load external entity "example.xml"

In der Vorlesung haben wir gesehen wie man XML auch programmatisch schreiben kann. Probieren Sie es hier aus indem Sie das Beispiel entsprechend dem vorherigen XML Dokument vervollständigen.

In [4]:
from lxml import etree as et

discography = et.Element('discography')
albums = et.SubElement(discography, 'albums')
album_1 = et.SubElement(albums, 'album')
album_1_title = et.SubElement(album_1, 'title')
album_1_title.text = 'The Dark Side of the Moon'
album_1_released = et.SubElement(album_1, 'released')
album_1_released.set('day', '16')
album_1_released.set('month', 'March')
album_1_released.set('year', '1973')
album_2 = et.SubElement(albums, 'album')
album_2_title = et.SubElement(album_2, 'title')
album_2_title.text = 'The Wall'
album_2_released = et.SubElement(album_2, 'released')
album_2_released_day = et.SubElement(album_2_released, 'day')
album_2_released_month = et.SubElement(album_2_released, 'month')
album_2_released_year = et.SubElement(album_2_released, 'year')
album_2_released_day.text = '30'
album_2_released_month.text = 'November'
album_2_released_year.text = '1979'

# Complete the example to match the previous XML document

print(et.tostring(discography, pretty_print=True).decode('utf-8'))

<discography>
  <albums>
    <album>
      <title>The Dark Side of the Moon</title>
      <released day="16" month="March" year="1973"/>
    </album>
    <album>
      <title>The Wall</title>
      <released>
        <day>30</day>
        <month>November</month>
        <year>1979</year>
      </released>
    </album>
  </albums>
</discography>



Schreiben Sie ihre discographie in eine Datei. Benennen Sie diese `discography.xml`. 

Schauen Sie im Verzeichnis nach ob die Datei korrekt gespeichert wurde.

In [7]:
from lxml import etree as et

t = et.ElementTree(discography)
# Edit the file name accordingly
t.write(file='discography.xml', pretty_print=True)

Im nächsten Beispiel travesieren wir unser XML Dokument rekursiv.

In [8]:
def expand(element):
    for child in element:
        if len(child) or child.text == None:
            print('{}'.format(child.tag))
        else:
            print('{}: {}'.format(child.tag, child.text))
        expand(child)
        
expand(discography)

albums
album
title: The Dark Side of the Moon
released
album
title: The Wall
released
day: 30
month: November
year: 1979


Das letzte Beispiel liest nun ein XML Dokument von einem Web Service und sucht nach einem bestimmten Element, wobei der Textinhalt ausgeschrieben wird. Dies ist die DOI `10.1594/PANGAEA.858171`.

Was geschieht in diesem Programmcode eigentlich? Benutzen Sie `print()` um das XML Dokument anzuschauen.

Was ist eine DOI? Fragen Sie Ihren Nachbarn oder suchen Sie im Internet danach.

Was ist [PANGAEA](https://pangaea.de)? Suchen Sie nach der DOI in PANGAEA. Was erhalten Sie?

Bilden Sie die `url` so, dass Sie diese in einem Browser ausführen können. Was erhalten Sie? Schauen Sie sich auch den `source` der Seite an.

In [23]:
import requests
from lxml import etree as et

url = '{}?verb={}&metadataPrefix={}&identifier={}'.format(
    'http://ws.pangaea.de/oai/provider',
    'GetRecord', 
    'datacite3', 
    'oai:pangaea.de:doi:10.1594/PANGAEA.858171'
)
    
r = requests.get(url)

x = et.XML(bytes(bytearray(r.text, 'utf-8')))

print(x.find('.//{http://datacite.org/schema/kernel-3}identifier[@identifierType="DOI"]').text)

10.1594/PANGAEA.858171


In [9]:
https://doi.pangaea.de/10.1594/PANGAEA.858171

SyntaxError: invalid syntax (<ipython-input-9-a053ae06d79e>, line 1)